### Semantic Control in Diffusion Inpainting: Merging RePaint Sampling with Text-Driven Generation in GLIDE

In [1]:
import numpy as np
import torch as torch
import gradio as gr

/home/infres/kbrowder-24/jp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(gr.__version__)

5.20.1


In [3]:
import pydantic
print(pydantic.__version__)

2.10.6


In [4]:
# This notebook supports both CPU and GPU.
# On CPU, generating one sample may take on the order of 20 minutes.
# On a GPU, it should be under a minute.

has_cuda = torch.cuda.is_available()
device = torch.device('cuda:2' if has_cuda else 'cpu')
print("Using device:", device)

Using device: cuda:2


## Data

For practical reasons, we run the notebook in the Google Drive folder where it is located. We need to provide a path.

In [5]:
# from google.colab import drive

# drive.mount('/content/drive')

# # Replace path_to_notebook with your actual path
# path_to_notebook = "drive/MyDrive/Colab Notebooks/M1 Data AI/Computer Vision/Project"

# %cd {path_to_notebook}

# !pwd
# !ls

### Sampling with Repaint sampling method

In [ ]:
import repaint_sampling as RS
import repaint_patcher as RP
import prepare_glide_inpaint as PGI
from image_util import *
import torch

In [47]:
model_base, diffusion_base, options_base = PGI.create_glide_generative(device=device, cuda=has_cuda)
model_rp, diffusion_rp, options_rp = PGI.create_glide_generative(device=device, cuda=has_cuda, use_inpaint=False)
model_rpip, diffusion_rpip, options_rpip = PGI.create_glide_generative(device=device, cuda=has_cuda)

RP.patch_model_for_repaint(diffusion_rp)
RP.patch_model_for_repaint(diffusion_rpip)
diffusion_base.p_sample_loop = torch.compile(diffusion_base.p_sample_loop, mode="max-autotune")
diffusion_rp.p_sample_loop = torch.compile(diffusion_rp.p_sample_loop, mode="max-autotune")
diffusion_rpip.p_sample_loop = torch.compile(diffusion_rpip.p_sample_loop, mode="max-autotune")


model_up_base, diffusion_up_base, options_up_base = PGI.create_glide_upsampler(device=device, cuda=has_cuda)
model_up_rp, diffusion_up_rp, options_up_rp = PGI.create_glide_upsampler(device=device, cuda=has_cuda, use_inpaint=False)
model_up_rpip, diffusion_up_rpip, options_up_rpip = PGI.create_glide_upsampler(device=device, cuda=has_cuda)

RP.patch_model_for_repaint(diffusion_up_rp)
RP.patch_model_for_repaint(diffusion_up_rpip)

sampler_up_base = RS.UpscaleSamplerInpaint(model_up_base, diffusion_up_base, options_up_base, model_fn=None, device=device)
sampler_up_base.sample = torch.compile(sampler_up_base.sample, mode="max-autotune")
sampler_up_rp = RS.UpscaleSamplerRepaint(model_up_rp, diffusion_up_rp, options_up_rp, model_fn=None, device=device)
sampler_up_rp.sample = torch.compile(sampler_up_rp.sample, mode="max-autotune")
sampler_up_rpip = RS.UpscaleSamplerRepaint(model_up_rpip, diffusion_up_rpip, options_up_rpip, model_fn=None, device=device)
sampler_up_rpip.sample = torch.compile(sampler_up_rpip.sample, mode="max-autotune")
upsample_temp = 0.997

/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/download.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return th.load(

In [65]:
def inpaint_image(input_image, mask_image, prompt, guidance_scale=7.0, size=256, batch_size=1, variant="base"):
    """Inpaint the masked region of the image based on the text prompt."""
    sampler = None
    if variant=="base":
        sampler = RS.CFGSamplerInpaint(model_base, diffusion_base, options_base, guidance_scale, device)
    elif variant=="repaint":
        sampler = RS.CFGSamplerRepaint(model_rp, diffusion_rp, options_rp, guidance_scale, device)
    elif variant=="repaint_inpaint":
        sampler = RS.CFGSamplerRepaintInpaint(model_rpip, diffusion_rpip, options_rpip, guidance_scale, device)
    else:
        raise Exception("Invalid variant choice")
    
    sampler_up = None
    if variant=="base":
        sampler_up = sampler_up_base
    elif variant=="repaint":
        sampler_up = sampler_up_rp
    elif variant=="repaint_inpaint":
        sampler_up = sampler_up_rp

    # Process input image
    image_tensor = process_image(input_image, size=64)
    image_tensor_full = process_image(input_image, size=size)

    # Handling the mask from the drawing canvas
    if len(mask_image.shape) == 3 and mask_image.shape[2] == 4:
        mask_array = mask_image[:, :, 3]
    elif len(mask_image.shape) == 2:
        mask_array = mask_image
    else:
        mask_array = np.mean(mask_image, axis=2) if mask_image.shape[2] == 3 else mask_image

    mask_tensor = process_mask(mask_array, size=64)
    mask_tensor_full = process_mask(mask_array, size=size)

    # In GLIDE's inpainting model, 0 means "inpaint this", 1 means "keep this"
    # But in our UI, white (255) means "inpaint this", so we invert the mask
    inpaint_mask = 1.0 - mask_tensor
    inpaint_mask_full = 1.0 - mask_tensor_full

    # Rest of your function remains the same
    # For RePaint, we need the ground truth image and keep mask
    gt = image_tensor
    gt_keep_mask = inpaint_mask  # 1 for areas to keep (not inpaint)

    gt_full = image_tensor_full
    gt_keep_mask_full = inpaint_mask_full

    # Sampling from the model
    print("Generating inpainted image...")

    jump_params = {
        "t_T": 250,
        "n_sample": 1,
        "jump_length": 10,
        "jump_n_sample": 3,
        "start_resampling": 20
    }
    samples = None
    if variant=="base":
        samples = sampler.sample(gt, gt_keep_mask, prompt, batch_size)[:batch_size]
    else:
        samples = sampler.sample(gt, gt_keep_mask, prompt, batch_size, jump_params=jump_params)[:batch_size]

    if variant=="base":
        samples = sampler_up.sample(samples, upsample_temp, gt_full, gt_keep_mask_full, prompt, batch_size)[:batch_size]
    else:
        samples = sampler_up.sample(samples, upsample_temp, gt_full, gt_keep_mask_full, prompt, batch_size, jump_params=jump_params)[:batch_size]

    # Convert the tensor to a numpy array in the range [0, 255]
    samples = ((samples + 1) * 127.5).clamp(0, 255).to(torch.uint8)
    samples = samples.permute(0, 2, 3, 1).contiguous()
    sample_image = samples[0].cpu().numpy()

    return sample_image

In [53]:
#warm up model
img = Image.open("data/datasets/places2/Places365_val_00000001.jpg")
img = img.resize((256, 256))
img = np.array(img)
mask = Image.open("data/masks/64/thick.png")
mask = mask.resize((256, 256), resample=Image.NEAREST)
mask = np.array(mask)
inpaint_image(img, mask, "prmpt", variant="base")
inpaint_image(img, mask, "prmpt", variant="repaint")
inpaint_image(img, mask, "prmpt", variant="repaint_inpaint")
print("Done Compiling Models")

Generating inpainted image...
CFGSampler


  0%|          | 0/250 [00:00<?, ?it/s]skipping cudagraphs due to skipping cudagraphs due to cpu device (arg789_1). Found from : 
   File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/gaussian_diffusion.py", line 340, in p_sample
    out = self.p_mean_variance(
  File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/respace.py", line 94, in p_mean_variance
    return super().p_mean_variance(self._wrap_model(model), *args, **kwargs)
  File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/gaussian_diffusion.py", line 242, in p_mean_variance
    min_log = _extract_into_tensor(self.posterior_log_variance_clipped, t, x.shape)
  File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/gaussian_diffusion.py", line 636, in _extract_into_tensor
    res = th.from_numpy(arr).to(device=timesteps.device)[times

Generating inpainted image...
CFGSampler
times: [249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 4

  0%|          | 0/452 [00:00<?, ?it/s]AUTOTUNE convolution(2x3x64x64, 192x3x3x3)
  convolution 0.0031 ms 100.0% 
  triton_convolution2d_33601 0.0154 ms 20.0% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=256, BLOCK_N=64, GROUPS=1, KERNEL_H=3, KERNEL_W=3, PADDING_H=1, PADDING_W=1, STRIDE_H=1, STRIDE_W=1, UNROLL=False, num_stages=2, num_warps=8
  triton_convolution2d_33596 0.0164 ms 18.8% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=256, BLOCK_N=64, GROUPS=1, KERNEL_H=3, KERNEL_W=3, PADDING_H=1, PADDING_W=1, STRIDE_H=1, STRIDE_W=1, UNROLL=False, num_stages=2, num_warps=4
  triton_convolution2d_33599 0.0164 ms 18.8% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, GROUPS=1, KERNEL_H=3, KERNEL_W=3, PADDING_H=1, PADDING_W=1, STRIDE_H=1, STRIDE_W=1, UNROLL=False, num_stages=2, num_warps=4
  triton_convolution2d_33597 0.0195 ms 15.8% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=1024, BLOCK_N=16, GROUPS=1, KERNEL_H=3, KERNEL_W=3, PADDING_H=1, PADDING_W=1, STRIDE_H=1, STRIDE_W=1, UNROLL=False, num_stages=1, num_wa

times: [249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39

...

Generating inpainted image...
CFGSampler
times: [249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 4

100%|██████████| 452/452 [00:17<00:00, 26.42it/s]


times: [249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39

100%|██████████| 452/452 [00:21<00:00, 21.28it/s]

Done Compiling Models


In [66]:
def process_input_with_mask(input_with_mask, prompt, guidance_scale, variant):
    """
    Process an image with drawing mask and extract both components for inpainting.

    Args:
        input_with_mask: Image with alpha channel containing mask information
        prompt: Text prompt for guiding the inpainting
        guidance_scale: Scale factor for classifier-free guidance

    Returns:
        Inpainted image or error message
    """
    print(variant)
    # return "function executed"
    # gr.Info(f"Input Image Dict: {input_with_mask.keys()}")
    # gr.Info(f"Background: {input_with_mask['background'].shape if isinstance(input_with_mask['background'], np.ndarray) else 'Invalid Image'}")
    # gr.Info(f"Layers: {len(input_with_mask['layers'])}")
    # gr.Info(f"Composite: {input_with_mask['composite'].shape if isinstance(input_with_mask['composite'], np.ndarray) else 'Invalid Image'}")
    # gr.Info(f"Layer 1: {input_with_mask['layers'][0].shape if isinstance(input_with_mask['layers'][0], np.ndarray) else 'Invalid Image'}")
    # return input_with_mask['layers'][0]


    # Check if there's any drawing on the image
    if input_with_mask is None:
        return "Please upload an image first."
    

    if len(input_with_mask['background'].shape) == 3:
        if len(input_with_mask['layers']) == 1:
            # The 4th channel contains the mask (alpha channel)
            original_image = input_with_mask['background'][:, :, :3]

            # Extract mask from alpha channel, white (255) marks areas to inpaint
            mask = input_with_mask['layers'][0]

            # Check if any masking was done
            if np.max(mask) == 0:
                return "Please draw on the image to create a mask for inpainting."

            # Process the image and mask for inpainting
            gr.Info(f"Original Image: {original_image.shape}")
            gr.Info(f"Mask: {mask.shape}")
            return inpaint_image(original_image, mask, prompt, guidance_scale, variant=variant, size=256)
        else:
            return "The image doesn't have an alpha channel for masking. Please draw on the image."
    else:
        return "Please upload a valid image and draw on it to create a mask."



### Gradio Interface

In [67]:
def create_gradio_interface():
    """Create the Gradio interface for the inpainting application."""
    with gr.Blocks(title="RePaint with GLIDE Text-Driven Inpainting") as app:
        gr.Markdown("# Text-Driven Image Inpainting with RePaint and GLIDE")
        gr.Markdown(
            "Upload an image, draw directly on it to create a mask (white areas will be inpainted), and enter a text prompt.")

        with gr.Row():
            with gr.Column():
                input_image = gr.ImageMask(label="Upload & Draw Mask (white areas will be inpainted)",
                                       image_mode="RGBA",
                                       type='numpy',
                                       sources=('upload'),	
                                       interactive=True)

                prompt = gr.Textbox(label="Text Prompt", placeholder="Describe what should be in the masked area...")
                guidance_scale = gr.Slider(minimum=1.0, maximum=15.0, value=7.0, step=0.5,
                                           label="Guidance Scale (higher = more text influence)")
                variant = gr.Radio([("Just Glide Inpainting", "base"),("Glide model with repaint strategy", "repaint"),("Glide Inpainting with repaint strategy", "repaint_inpaint")],value="base")
                submit_btn = gr.Button("Generate Inpainting")

            with gr.Column():
                output_image = gr.Image(label="Inpainted Result")

        submit_btn.click(
            fn=process_input_with_mask,
            inputs=[input_image, prompt, guidance_scale, variant],
            outputs=output_image,
        )

        gr.Markdown("## How to use")
        gr.Markdown("""
        1. Upload an image
        2. Draw directly on the image with white brush to create a mask (white areas will be inpainted)
        3. Enter a text prompt describing what you want in the masked area
        4. Click 'Generate Inpainting' and wait for the result
        5. Adjust the guidance scale if needed (higher values follow the text more closely)

        Tips:
        - Use a larger brush radius for covering larger areas
        - You can clear your drawing and start over using the clear button
        - Be specific in your text prompt for best results
        """)

    return app

In [68]:
import os

demo = create_gradio_interface()
os.makedirs("sample_images", exist_ok=True)
demo.launch(share=True, debug=True, inline=True, show_error=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://50636c1d24b1675b44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


base
Generating inpainted image...
CFGSampler


  0%|          | 0/250 [00:00<?, ?it/s]skipping cudagraphs due to skipping cudagraphs due to cpu device (arg793_1). Found from : 
   File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/gaussian_diffusion.py", line 340, in p_sample
    out = self.p_mean_variance(
  File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/respace.py", line 94, in p_mean_variance
    return super().p_mean_variance(self._wrap_model(model), *args, **kwargs)
  File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/gaussian_diffusion.py", line 242, in p_mean_variance
    min_log = _extract_into_tensor(self.posterior_log_variance_clipped, t, x.shape)
  File "/home/infres/kbrowder-24/jp/CSC_52002_EP_Generative_AI_Project/glide-text2im-main/glide_text2im/gaussian_diffusion.py", line 636, in _extract_into_tensor
    res = th.from_numpy(arr).to(device=timesteps.device)[times

base
Generating inpainted image...
CFGSampler


100%|██████████| 250/250 [00:07<00:00, 32.29it/s]


...

repaint_inpaint
Generating inpainted image...
CFGSampler
times: [249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 5

100%|██████████| 452/452 [00:17<00:00, 25.97it/s]


times: [249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39

100%|██████████| 452/452 [00:21<00:00, 21.00it/s]


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://50636c1d24b1675b44.gradio.live
